In [20]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re
from tensorflow.keras.models import load_model

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv(r'C:\Users\alekh\Downloads\Data.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)
model.save("final_model.h5")


<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
C:\Users\alekh\AppData\Local\Temp\ipykernel_22268\3001368605.py:19: SyntaxWarning: invalid escape sequence '\s'
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
C:\Users\alekh\AppData\Local\Temp\ipykernel_22268\3001368605.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
C:\Users\alekh\AppData\Local\Temp\ipykernel_22268\3001368605.py:22: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')
C:\Users\alekh\anaconda3\L

291/291 - 29s - 100ms/step - accuracy: 0.6396 - loss: 0.8405
144/144 - 5s - 35ms/step - accuracy: 0.6606 - loss: 0.7633


0.7632901072502136
0.6605504751205444
['loss', 'compile_metrics']


In [22]:
import numpy as np

# Load saved model and tokenizer
model = load_model("final_model.h5")

# New input
new_text = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing .@realDonaldTrump"]
new_text = [re.sub('[^a-zA-z0-9\s]', '', x.lower()) for x in new_text]
seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=X.shape[1])
pred = model.predict(padded)
pred_label = labelencoder.inverse_transform([np.argmax(pred)])

print("Prediction:", pred_label[0])

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\alekh\AppData\Local\Temp\ipykernel_22268\2478378491.py:8: SyntaxWarning: invalid escape sequence '\s'
  new_text = [re.sub('[^a-zA-z0-9\s]', '', x.lower()) for x in new_text]


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: Negative


In [47]:
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

data = pd.read_csv(r'C:\Users\alekh\Downloads\Data.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

def create_model(optimizer='adam'):
        model = Sequential()
        model.add(Dense(neurons, activation='relu', input_dim=10))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return model    
    
#Wrapping with Keras Classifier
classifier = KerasClassifier(model=create_model, verbose=0)

param_grid = {
    'batch_size': [16, 32],
    'epochs': [3]
    }
grid = GridSearchCV(estimator = classifier, param_grid = param_grid, scoring = 'accuracy', cv = 3)
grid_result = grid.fit(X,y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
C:\Users\alekh\AppData\Local\Temp\ipykernel_22268\225373131.py:10: SyntaxWarning: invalid escape sequence '\s'
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
C:\Users\alekh\AppData\Local\Temp\ipykernel_22268\225373131.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
C:\Users\alekh\AppData\Local\Temp\ipykernel_22268\225373131.py:13: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')
C:\Users\alekh\AppData\Local\

ValueError: 
All the 6 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\alekh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\alekh\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "C:\Users\alekh\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "C:\Users\alekh\anaconda3\Lib\site-packages\scikeras\wrappers.py", line 925, in _fit
    X, y = self._initialize(X, y)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alekh\anaconda3\Lib\site-packages\scike

In [31]:
pip install scikeras

Note: you may need to restart the kernel to use updated packages.
